# UK - Anbindung Paid Themen

* Um was handelt es sich hier  (Kurzbeschreibung Inhalt):
- Die Paid Data Transformatation von Meta


---
* QUELLEN:  
- datif_pz_uk_{}.03_transformed.meta_paid_daily
- datif_pz_uk_{}.03_transformed.meta_paid_total

* ZIEL:  
- datif_pz_uk_{}.03_transformed.meta_paid_organic_daily
- datif_pz_uk_{}.03_transformed.meta_paid_organic_total

---
* Versionen (aktuelle immer oben):
- 18.09.2025 Max Mustermann: init


# 1. Initialnotebooks & Libraries

In [0]:
%run ../../common/nb_init

## 03-transformed

In [0]:
target_schema_name = "03_transformed"
target_path = "funnel"

### Meta daily

In [0]:
spark.sql(f"""
    CREATE OR REPLACE VIEW datif_pz_uk_{env}.03_transformed.meta_paid_organic_daily AS
    SELECT
        CASE
                WHEN io.PostID IS NULL and fo.PostID IS NULL THEN 'Paid'
                WHEN mp.AdID IS NULL THEN 'Organic'
                ELSE 'Organic&Paid'
            END AS Source,
        CASE
                WHEN io.PostID IS NOT NULL THEN 'Instagram'
                WHEN fo.PostID IS NOT NULL THEN 'Facebook'
                ELSE 'Meta'
            END AS Plattform,
        COALESCE(io.Date, fo.Date, mp.Date, 'Existiert nicht') AS Date,
        COALESCE(io.CreatedDate, fo.CreatedDate, 'Existiert nicht') AS CreatedDate,
        COALESCE(io.PostID, fo.PostID,'Existiert nicht') AS OrganicID,
        COALESCE(io.PostURL, fo.PostURL, 'Existiert nicht') AS URL,
        COALESCE(io.PostType, fo.PostType, 'Existiert nicht') AS PostType,
        COALESCE(
            io.TotalImpressions,
            CASE
                WHEN fo.PostTotalImpressionsLifetime IS NOT NULL 
                    AND mp.Impressions IS NOT NULL THEN fo.PostTotalImpressionsLifetime - mp.Impressions
                WHEN fo.PostTotalImpressionsLifetime IS NOT NULL THEN fo.PostTotalImpressionsLifetime
                ELSE 0
            END
        ) AS Organic_Impressions,

        -- Organic Engagements
        COALESCE(
            io.Engagement,
            Case
                WHEN fo.Engagement IS NOT NULL
                    AND mp.Engagements IS NOT NULL THEN fo.Engagement - mp.Engagements
                WHEN fo.Engagement IS NOT NULL THEN fo.Engagement
                ELSE 0
            END
        ) AS Organic_Engagements,

        -- Organic Weighted Engagements
        COALESCE(
            round(io.WeightedEngagements,2),
            CASE
                WHEN fo.WeightedEngagements IS NOT NULL
                    AND mp.WeightedEngagements IS NOT NULL THEN round(fo.WeightedEngagements - mp.WeightedEngagements,2)
                WHEN fo.WeightedEngagements IS NOT NULL THEN round(fo.WeightedEngagements,2)
                ELSE 0
            END
        ) AS Organic_WeightedEngagements,
        
        -- Organic Shares
        COALESCE(
            io.TotalSharesDiff,
            CASE
                WHEN fo.TotalPostSharesDiff IS NOT NULL
                     AND mp.Post_Shares IS NOT NULL THEN fo.TotalPostSharesDiff - mp.Post_Shares
                WHEN fo.TotalPostSharesDiff IS NOT NULL THEN fo.TotalPostSharesDiff
                ELSE 0
            END
        ) AS Organic_Shares,

        -- Organic Clicks
        COALESCE(
            io.TotalInteractionsDiff,
            CASE
                WHEN fo.LinkClicksLifetimeDiff IS NOT NULL
                     AND mp.Link_Clicks IS NOT NULL THEN fo.LinkClicksLifetimeDiff - mp.Link_Clicks
                WHEN fo.LinkClicksLifetimeDiff IS NOT NULL THEN fo.LinkClicksLifetimeDiff
                ELSE 0
            END
        ) AS Organic_Clicks,

        -- Organic Comments
        COALESCE(
            io.TotalCommentsDiff,
            CASE
                WHEN fo.TotalPostCommentsDiff IS NOT NULL
                     AND mp.Comments IS NOT NULL THEN fo.TotalPostCommentsDiff - mp.Comments
                WHEN fo.TotalPostCommentsDiff IS NOT NULL THEN fo.TotalPostCommentsDiff
                ELSE 0
            END
        ) AS Organic_Comments,

        -- Organic Reactions
        COALESCE(
            io.TotalLikesDiff,
            CASE
                WHEN fo.TotalReactionsDiff IS NOT NULL
                     AND mp.Reactions IS NOT NULL THEN fo.TotalReactionsDiff - mp.Reactions
                WHEN fo.TotalReactionsDiff IS NOT NULL THEN fo.TotalReactionsDiff
                ELSE 0
            END
        ) AS Organic_Reactions,
        COALESCE(round(io.TotalSavedDiff,2), 0) AS Organic_Saved,
        COALESCE(mp.AdID, 'Existiert nicht') AS AdID,
        COALESCE(mp.CampaignName, 'Existiert nicht') AS CampaignName,
        COALESCE(mp.AdSetName, 'Existiert nicht') AS AdSetName,
        COALESCE(mp.AdName, 'Existiert nicht') AS AdName,
        COALESCE(round(mp.Amount_Spend,2), 0) AS Amount_Spend,
        COALESCE(mp.Impressions, 0) AS Paid_Impressions,
        COALESCE(mp.Engagements, 0) AS Paid_Engagements,
        COALESCE(round(mp.WeightedEngagements,2), 0) AS Paid_WeightedEngagements,
        COALESCE(round(mp.Post_Shares,2), 0) AS Paid_Shares,
        COALESCE(round(mp.Link_Clicks,2), 0) AS Paid_Clicks,
        COALESCE(round(mp.Comments,2), 0) AS Paid_Comments,
        COALESCE(round(mp.Reactions,2), 0) AS Paid_Reactions,
        COALESCE(Organic_Impressions + Paid_Impressions, 0) AS All_Impressions,
        COALESCE(Organic_Engagements + Paid_Engagements, 0) AS All_Engagements,
        COALESCE(Organic_WeightedEngagements + Paid_WeightedEngagements, 0) AS All_WeightedEngagements,
        COALESCE(Organic_Shares + Paid_Shares, 0) AS All_Shares,
        COALESCE(Organic_Clicks + Paid_Clicks, 0) AS All_Clicks,
        COALESCE(Organic_Comments + Paid_Comments, 0) AS All_Comments,
        COALESCE(Organic_Reactions + Paid_Reactions, 0) AS All_Reactions
    FROM datif_pz_uk_{env}.03_transformed.instagram_organic_post_daily AS io
    FULL OUTER JOIN datif_pz_uk_{env}.03_transformed.facebook_organic_daily AS fo
        ON io.PostID = fo.PostID
    FULL OUTER JOIN datif_pz_uk_{env}.03_transformed.meta_paid_daily AS mp
        ON (
            ((io.PostID = mp.OrganicID) AND CAST(io.Date AS DATE) = CAST(mp.Date AS DATE))
            OR
            (fo.PostID LIKE CONCAT('%', mp.OrganicID, '%') AND CAST(fo.Date AS DATE) = CAST(mp.Date AS DATE))
        )
""")


### Meta Total

In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW datif_pz_uk_{env}.03_transformed.meta_paid_organic_total AS
SELECT
    CASE
        WHEN io.PostID IS NULL AND fo.PostID IS NULL THEN 'Paid'
        WHEN mp.ID IS NULL THEN 'Organic'
        ELSE 'Organic&Paid'
    END AS Source,

    CASE
        WHEN io.PostID IS NOT NULL THEN 'Instagram'
        WHEN fo.PostID IS NOT NULL THEN 'Facebook'
        ELSE 'Meta'
    END AS Plattform,

    COALESCE(io.PostID, fo.PostID, 'Existiert nicht') AS OrganicID,
    COALESCE(io.CreatedDate, fo.CreatedDate, 'Existiert nicht') AS Organic_CreatedDate,
    COALESCE(mp.CreatedDate, 'Existiert nicht') AS Paid_CreatedDate,
    COALESCE(io.PostURL, fo.PostURL, 'Existiert nicht') AS URL,
    COALESCE(io.PostType, fo.PostType, 'Existiert nicht') AS PostType,
    COALESCE(io.PostMessage, fo.PostMessage, 'Existiert nicht') AS PostMessage,



    -- Organische KPIs
    COALESCE(
            io.TotalViews,
            CASE
                WHEN fo.PostTotalImpressionsLifetime IS NOT NULL 
                    AND mp.TotalImpressions IS NOT NULL THEN fo.PostTotalImpressionsLifetime - mp.TotalImpressions
                WHEN fo.PostTotalImpressionsLifetime IS NOT NULL THEN fo.PostTotalImpressionsLifetime
                ELSE 0
            END
        ) AS Organic_TotalImpressions,

    -- Engagements
    COALESCE(
        io.TotalEngagements,
        CASE
            WHEN fo.TotalEngagements IS NOT NULL
                 AND mp.TotalEngagements IS NOT NULL THEN fo.TotalEngagements - mp.TotalEngagements
            WHEN fo.TotalEngagements IS NOT NULL THEN fo.TotalEngagements
            ELSE 0
        END
    ) AS Organic_TotalEngagements,

    -- Weighted Engagements (gewichtete Interaktionen)
    COALESCE(
        ROUND(io.TotalWeightedEngagements, 2),
        CASE
            WHEN fo.TotalWeightedEngagements IS NOT NULL
                 AND mp.TotalWeightedEngagements IS NOT NULL THEN ROUND(fo.TotalWeightedEngagements - mp.TotalWeightedEngagements, 2)
            WHEN fo.TotalWeightedEngagements IS NOT NULL THEN ROUND(fo.TotalWeightedEngagements, 2)
            ELSE 0
        END
    ) AS Organic_TotalWeightedEngagements,

        -- Shares
    COALESCE(
        io.TotalShares,
        CASE
            WHEN fo.TotalPostShares IS NOT NULL
                 AND mp.TotalPostShares IS NOT NULL THEN fo.TotalPostShares - mp.TotalPostShares
            WHEN fo.TotalPostShares IS NOT NULL THEN fo.TotalPostShares
            ELSE 0
        END
    ) AS Organic_TotalShares,

    -- Clicks
    COALESCE(
        io.TotalInteractions,
        CASE
            WHEN fo.LinkClicksLifetime IS NOT NULL
                 AND mp.TotalLinkClicks IS NOT NULL THEN fo.LinkClicksLifetime - mp.TotalLinkClicks
            WHEN fo.LinkClicksLifetime IS NOT NULL THEN fo.LinkClicksLifetime
            ELSE 0
        END
    ) AS Organic_TotalClicks,

    -- Comments
    COALESCE(
        io.TotalComments,
        CASE
            WHEN fo.TotalPostComments IS NOT NULL
                 AND mp.TotalPostComments IS NOT NULL THEN fo.TotalPostComments - mp.TotalPostComments
            WHEN fo.TotalPostComments IS NOT NULL THEN fo.TotalPostComments
            ELSE 0
        END
    ) AS Organic_TotalComments,

    -- Reactions
    COALESCE(
        io.TotalLikes,
        CASE
            WHEN (
                fo.TotalLikeReactions + fo.TotalAngerReactions + fo.TotalhahaReactions +
                fo.TotalLoveReactions + fo.TotalSadReactions + fo.TotalwowReactions
            ) IS NOT NULL
            AND mp.TotalReactions IS NOT NULL THEN
                (fo.TotalLikeReactions + fo.TotalAngerReactions + fo.TotalhahaReactions +
                 fo.TotalLoveReactions + fo.TotalSadReactions + fo.TotalwowReactions) - mp.TotalReactions
            WHEN (
                fo.TotalLikeReactions + fo.TotalAngerReactions + fo.TotalhahaReactions +
                fo.TotalLoveReactions + fo.TotalSadReactions + fo.TotalwowReactions
            ) IS NOT NULL THEN
                (fo.TotalLikeReactions + fo.TotalAngerReactions + fo.TotalhahaReactions +
                 fo.TotalLoveReactions + fo.TotalSadReactions + fo.TotalwowReactions)
            ELSE 0
        END
    ) AS Organic_TotalReactions,
    COALESCE(ROUND(io.TotalSaved, 2), 0) AS Organic_TotalSaved,

    -- Engagement Rate
    COALESCE(
        io.EngagementRateInPercent,
        CASE
            WHEN fo.EngagementRateInPercent IS NOT NULL 
                And mp.EngagementRatings IS NOT NULL THEN ROUND(fo.EngagementRateInPercent - mp.EngagementRatings,2)
            WHEN fo.EngagementRateInPercent IS NOT NULL THEN fo.EngagementRateInPercent
            ELSE 0
        END
    ) AS Organic_EngagementRateInPercent,

    -- Paid-Daten
    COALESCE(mp.ID, 'Existiert nicht') AS AdID,
    COALESCE(mp.CampaignName, 'Existiert nicht') AS CampaignName,
    COALESCE(mp.AdName, 'Existiert nicht') AS AdName,
    COALESCE(ROUND(mp.TotalSpend, 2), 0) AS TotalSpend,
    COALESCE(mp.TotalImpressions, 0) AS Paid_TotalImpressions,
    COALESCE(mp.TotalEngagements, 0) AS Paid_TotalEngagements,
    COALESCE(ROUND(mp.TotalWeightedEngagements, 2), 0) AS Paid_TotalWeightedEngagements,
    COALESCE(mp.EngagementRatings, 0) AS Paid_TotalEngagementRating,
    COALESCE(ROUND(mp.TotalPostShares, 2), 0) AS Paid_TotalShares,
    COALESCE(ROUND(mp.TotalLinkClicks, 2), 0) AS Paid_TotalClicks,
    COALESCE(ROUND(mp.TotalPostComments, 2), 0) AS Paid_TotalComments,
    COALESCE(ROUND(mp.TotalReactions, 2), 0) AS Paid_TotalReactions,
    COALESCE(mp.CPM, 0) AS CPM,
    COALESCE(mp.CPC, 0) AS CPC,
    COALESCE(mp.CTR, 0) AS CTR,
    COALESCE(mp.CPV, 0) AS CTV,
    COALESCE(mp.VideoViewRate, 0) AS VideoViewRate,

        -- Gesamtergebnisse (kombiniert)
    COALESCE(Paid_TotalImpressions + Organic_TotalImpressions, 0) AS All_TotalImpressions,
    COALESCE(Paid_TotalEngagements + Organic_TotalEngagements, 0) AS All_TotalEngagements,
    COALESCE(Paid_TotalWeightedEngagements + Organic_TotalWeightedEngagements,0) AS All_TotalWeightedEngagements,
    COALESCE(Paid_TotalShares + Organic_TotalShares, 0) AS All_TotalShares,
    COALESCE(Paid_TotalClicks + Organic_TotalClicks, 0) AS All_TotalClicks,
    COALESCE(Paid_TotalComments + Organic_TotalComments, 0) AS All_TotalComments,
    COALESCE(Paid_TotalReactions + Organic_TotalReactions, 0) AS All_TotalReactions,
    COALESCE(Organic_TotalSaved, 0) AS All_TotalSaved,
    COALESCE(Organic_EngagementRateInPercent + Paid_TotalEngagementRating, 0) AS All_EngagementRateInPercent,

    -- Neue Themen-Spalten aus beiden organischen Quellen
    COALESCE(io.Themenbereich1, fo.Themenbereich1, 'Existiert nicht') AS Themenbereich1,
    COALESCE(io.Themenbereich2, fo.Themenbereich2, 'Existiert nicht') AS Themenbereich2,
    COALESCE(io.Themenbereich3, fo.Themenbereich3, 'Existiert nicht') AS Themenbereich3,
    COALESCE(io.Themenbereich1_Conf, fo.Themenbereich1_Conf, 0) AS Themenbereich1_Conf,
    COALESCE(io.Themenbereich2_Conf, fo.Themenbereich2_Conf, 0) AS Themenbereich2_Conf,
    COALESCE(io.Themenbereich3_Conf, fo.Themenbereich3_Conf, 0) AS Themenbereich3_Conf,
    COALESCE(io.Strategie2030, fo.Strategie2030, 0.0) AS Strategie2030,
    COALESCE(io.FinanzierungEnergiewende, fo.FinanzierungEnergiewende, 0.0) AS FinanzierungEnergiewende,
    COALESCE(io.EMobilitaet, fo.EMobilitaet, 0.0) AS EMobilitaet,
    COALESCE(io.VernetzeEnergiewelt, fo.VernetzeEnergiewelt, 0.0) AS VernetzeEnergiewelt,
    COALESCE(io.TransformationGasnetzeWasserstoff, fo.TransformationGasnetzeWasserstoff, 0.0) AS TransformationGasnetzeWasserstoff,
    COALESCE(io.ErneuerbareEnergien, fo.ErneuerbareEnergien, 0.0) AS ErneuerbareEnergien,
    COALESCE(io.DisponibleErzeugung, fo.DisponibleErzeugung, 0.0) AS DisponibleErzeugung,
    COALESCE(io.IntelligenteStromnetze, fo.IntelligenteStromnetze, 0.0) AS IntelligenteStromnetze,
    COALESCE(io.AAlsArbeitgeberIn, fo.AAlsArbeitgeberIn, 0.0) AS AAlsArbeitgeberIn,
    COALESCE(io.NachhaltigkeitCSRESG, fo.NachhaltigkeitCSRESG, 0.0) AS NachhaltigkeitCSRESG,
    COALESCE(io.MarkeA, fo.MarkeA, 0.0) AS MarkeA,
    COALESCE(io.CLevelErwaehnungen, fo.CLevelErwaehnungen, 0) AS CLevelErwaehnung

FROM datif_pz_uk_{env}.03_transformed.instagram_organic_total AS io
FULL OUTER JOIN datif_pz_uk_{env}.03_transformed.facebook_organic_total AS fo
    ON io.PostID = fo.PostID
FULL OUTER JOIN datif_pz_uk_{env}.03_transformed.meta_paid_total AS mp
    ON (
        (io.PostID = mp.OrganicID)
        OR
        (fo.PostID LIKE CONCAT('%', mp.OrganicID, '%'))
    )
""")
